In [ ]:
import dask
import numpy as np
import xarray as xr

from dask import distributed
from scipy.stats import norm

In [ ]:
def generate_diffusivity_function(n, x, offset=0.5):
    cdf_values = np.zeros(len(x))
    for _ in range(n):
        mean = np.max(x) * np.random.rand()
        variance = 0.05  * np.random.rand() + 0.01
        cdf_values = cdf_values + norm.pdf(x, loc=mean, scale=np.sqrt(variance))
    cdf_values = cdf_values / n
    return cdf_values + offset


def solve_heat_equation(u, alpha_x, dt, dx, Nt, T0, TL):    
    Nx = len(u)
    for _ in range(Nt):
        A = np.zeros((Nx-2, Nx-2))
        b = np.zeros(Nx-2)

        for i in range(1, Nx-1):
            alpha_avg_left = 0.5 * (alpha_x[i-1] + alpha_x[i])
            alpha_avg_right = 0.5 * (alpha_x[i] + alpha_x[i+1])

            A[i-1, i-1] = 1 + dt/dx**2 * (alpha_avg_left + alpha_avg_right)/2
            if i != 1:
                A[i-1, i-2] = -alpha_avg_left * dt/(2*dx**2)
            if i != Nx-2:
                A[i-1, i] = -alpha_avg_right * dt/(2*dx**2)

            b[i-1] = (
                u[i] 
                + alpha_x[i+1] * u[i+1] * dt/(2*dx**2) 
                + alpha_x[i-1] * u[i-1] * dt/(2*dx**2)
                - 2 * alpha_x[i] * u[i] * dt/(2*dx**2)
            )
        b[0] += alpha_x[0] * T0 * dt/(2*dx**2)
        b[-1] += alpha_x[-1] * TL * dt/(2*dx**2)
        u[1:-1] = np.linalg.solve(A, b)
    return u


def generate_training_sample(
    max_diffusivity_components=2,
    L=1.0,
    Nx=100,
    Tmax=10,
    dt=0.001,
):
    dx = L / (Nx-1) 
    Nt = int(Tmax / dt) 
    x = np.linspace(0, L, Nx)

    n = np.random.choice(np.arange(1, max_diffusivity_components+1))
    d = generate_diffusivity_function(n, x)
    T0 = d.min()
    TL = d.max()
    u = np.ones(Nx) * T0
    u[0], u[-1] = T0, TL

    u = solve_heat_equation(u, d, dt, dx, Nt, T0, TL)
    return u, d

def generate_training_data(N=1000, workers=64):
    cluster = distributed.LocalCluster(n_workers=workers, threads_per_worker=2)
    client = distributed.Client(cluster)

    delayed_fun = dask.delayed(generate_training_sample)
    results = dask.compute([delayed_fun() for _ in range(N)])[0]
    u_values, d_values = zip(*results)

    u_values = xr.DataArray(np.array(u_values), dims=('sample', 'x'), name='u')
    d_values = xr.DataArray(np.array(d_values), dims=('sample', 'x'), name='d')
    return xr.Dataset({'u': u_values, 'd': d_values})


In [ ]:
ds = generate_training_data(N=3000).astype(np.float32)
ds.to_netcdf('training_data.nc')

In [ ]:
ds = generate_training_data(N=1000).astype(np.float32)
ds.to_netcdf('validation_data.nc')

In [ ]:
ds = generate_training_data(N=1000).astype(np.float32)
ds.to_netcdf('testing_data.nc')